# Risk Flow Matrix Modeling

In [26]:
import pandas as pd
import numpy as np
import math

## Data Preprocessing

### Read the preprocessed dataset:
 - "origin_dest_trips_census_tract_level.csv"
 - "tracts_4.csv"

In [27]:
# this dataset is for the nested hashmap
OD_census_tract_level = pd.read_csv('origin_dest_trips_census_tract_level.csv')
OD_census_tract_level

,RESIDENCE,WORKPLACE,JOBS
0,17001000100,17001000100,139
1,17001000100,17001000201,12
2,17001000100,17001000202,9
3,17001000100,17001000400,10
4,17001000100,17001000500,119
...,...,...,...
80423,17203030700,17113005102,2
80424,17203030700,17143001600,5
80425,17203030700,17179021201,2
80426,17203030700,17203030601,6


In [28]:
tracts = pd.read_csv('tracts_4.csv')
tracts
# tracts = tracts.drop(columns = ('Unnamed: 0'))

,GEOID,ZIP_CODES,TRIPS_ORIG,TRIPS_DEST,POPULATION,STORES,CBSA_EMP,CBSA_POP,CBSA_WRK,COUNTHU10,...,No. Populated Places,No. Stores,Total Points of Interest,POP_DENSITY,NUM_WORKERS,EMPLOYMENT_DENSITY,PREDICTED_ORIG_TRIPS,PREDICTED_DEST_TRIPS,FIPS,avg_cases_per_tract
0,17091011700,0,27.0,111.0,3417,0,43299.0,113449.0,46799.0,378.250000,...,0.0,0,8.0,7.059873,46799.0,2.442126,126.347,125.582,17091,63.068966
1,17091011800,0,93.0,141.0,2627,0,43299.0,113449.0,46799.0,589.000000,...,1.0,0,3.0,5.838794,46799.0,1.327268,116.855,155.451,17091,63.068966
2,17119400951,0,433.0,299.0,4966,0,1261547.0,2812896.0,1237055.0,749.333333,...,0.0,0,4.0,3.772428,1237055.0,0.566353,314.676,283.279,17119,46.803279
3,17119400952,0,4.0,155.0,3335,0,1261547.0,2812896.0,1237055.0,712.000000,...,1.0,0,2.0,2.777786,1237055.0,0.242828,60.534,203.733,17119,46.803279
4,17135957500,['62533' '62560'],34.0,45.0,3273,0,0.0,0.0,0.0,372.000000,...,7.0,0,49.0,0.128183,0.0,0.040990,41.602,70.198,17135,21.875000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3118,17037000100,['60135'],145.0,245.0,6712,0,4161510.0,9461105.0,4066635.0,852.666667,...,2.0,0,18.0,1.022809,4066635.0,0.274031,160.356,242.515,17037,45.714286
3119,17037001500,0,346.0,156.0,3807,0,4161510.0,9461105.0,4066635.0,551.666667,...,1.0,0,8.0,5.213170,4066635.0,1.535521,404.125,199.039,17037,45.714286
3120,17037000400,['60178'],13.0,499.0,8622,2,4161510.0,9461105.0,4066635.0,1652.500000,...,4.0,2,24.0,0.252156,4066635.0,0.009529,193.709,440.224,17037,45.714286
3121,17037000300,['60111' '60150'],110.0,59.0,2680,0,4161510.0,9461105.0,4066635.0,552.000000,...,3.0,0,33.0,0.037200,4066635.0,0.012366,87.904,65.580,17037,45.714286


### Create a dictionary: to record each origin->destination: number of trips

use the dataset: OD_census_tract_level 

1. Generate a unique set of origins in commute trips

In [29]:
origins = OD_census_tract_level.loc[:,'RESIDENCE'].unique().tolist()
#origins

2. Generate a doubly nested hashmap (python dictionary: key-value pair): \
 first mapping: **origin -> destination** \
 second mapping: **destination -> number of trips**

In [30]:
i = 0
OD = OD_census_tract_level
mRes = {}
for origin in origins:
    mWork = {}
    while i < OD.shape[0] and OD.iloc[i,0] == origin:
        workplace = OD.iloc[i,1]
        jobs = OD.iloc[i,2]
        mWork[workplace] = jobs
        i += 1
    mRes[origin] = mWork

In [31]:
# the hashmap
# mRes

### Precess the tracts data (GEOID, TRIPS_ORIG, CASES)

use the dataset: tracts

In [32]:
# the origianl dataset read from csv file
tracts

,GEOID,ZIP_CODES,TRIPS_ORIG,TRIPS_DEST,POPULATION,STORES,CBSA_EMP,CBSA_POP,CBSA_WRK,COUNTHU10,...,No. Populated Places,No. Stores,Total Points of Interest,POP_DENSITY,NUM_WORKERS,EMPLOYMENT_DENSITY,PREDICTED_ORIG_TRIPS,PREDICTED_DEST_TRIPS,FIPS,avg_cases_per_tract
0,17091011700,0,27.0,111.0,3417,0,43299.0,113449.0,46799.0,378.250000,...,0.0,0,8.0,7.059873,46799.0,2.442126,126.347,125.582,17091,63.068966
1,17091011800,0,93.0,141.0,2627,0,43299.0,113449.0,46799.0,589.000000,...,1.0,0,3.0,5.838794,46799.0,1.327268,116.855,155.451,17091,63.068966
2,17119400951,0,433.0,299.0,4966,0,1261547.0,2812896.0,1237055.0,749.333333,...,0.0,0,4.0,3.772428,1237055.0,0.566353,314.676,283.279,17119,46.803279
3,17119400952,0,4.0,155.0,3335,0,1261547.0,2812896.0,1237055.0,712.000000,...,1.0,0,2.0,2.777786,1237055.0,0.242828,60.534,203.733,17119,46.803279
4,17135957500,['62533' '62560'],34.0,45.0,3273,0,0.0,0.0,0.0,372.000000,...,7.0,0,49.0,0.128183,0.0,0.040990,41.602,70.198,17135,21.875000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3118,17037000100,['60135'],145.0,245.0,6712,0,4161510.0,9461105.0,4066635.0,852.666667,...,2.0,0,18.0,1.022809,4066635.0,0.274031,160.356,242.515,17037,45.714286
3119,17037001500,0,346.0,156.0,3807,0,4161510.0,9461105.0,4066635.0,551.666667,...,1.0,0,8.0,5.213170,4066635.0,1.535521,404.125,199.039,17037,45.714286
3120,17037000400,['60178'],13.0,499.0,8622,2,4161510.0,9461105.0,4066635.0,1652.500000,...,4.0,2,24.0,0.252156,4066635.0,0.009529,193.709,440.224,17037,45.714286
3121,17037000300,['60111' '60150'],110.0,59.0,2680,0,4161510.0,9461105.0,4066635.0,552.000000,...,3.0,0,33.0,0.037200,4066635.0,0.012366,87.904,65.580,17037,45.714286


In [33]:
# rename columns
tracts = tracts.rename(columns = {'avg_cases_per_tract' : 'CASES'})
# select following columns
### what is 'TRIPS_ORIG'???
tracts = tracts.loc[:,('GEOID','TRIPS_ORIG','CASES')]
# convert the type to int
tracts.loc[:,'GEOID'] = tracts.loc[:,'GEOID'].astype(int)

In [34]:
tracts

,GEOID,TRIPS_ORIG,CASES
0,17091011700,27.0,63.068966
1,17091011800,93.0,63.068966
2,17119400951,433.0,46.803279
3,17119400952,4.0,46.803279
4,17135957500,34.0,21.875000
...,...,...,...
3118,17037000100,145.0,45.714286
3119,17037001500,346.0,45.714286
3120,17037000400,13.0,45.714286
3121,17037000300,110.0,45.714286


## Build Risk Flow Model

Formula: \
risk from census tract from i to j: \
case_number_at_i * num_of_trips_from_i_to_j / sum_of_product_of_cases_at_j_and_flux_from_j_for_all_j's

The risk of importation of COVID-19 cases in a country outside China ($\alpha$), from a city in China ($i$)

$r_{i\alpha}$ is the risk flow matrix

$$ r_{i\alpha} = \frac{e_i n_i }{\Sigma_je_jn_j} P_{i\alpha}$$

- where $r_i$ is the cumulttive incidence in city $i$ 
        (assumed to be homogeneous within each province)
- $n_i$ is the travel flux from $i$
- $P_{i\alpha}$ is the probability of traveling from city $i$ to country $\alpha$



In [35]:
# fill the missing value with 0
tracts.loc[:,'TRIPS_ORIG'] = tracts.loc[:,'TRIPS_ORIG'].fillna(0)
tracts.loc[:,'CASES'] = tracts.loc[:,'CASES'].fillna(0)

# calulate total risk flow:
# sum of the "TRIPS_ORIG" * "CASES" for each "GEOID"
total_risk_flow = sum([tracts.iloc[i, -1] * tracts.iloc[i, -2] for i in range(len(tracts))])
total_risk_flow

39555573.88410911

In [36]:
# populate horizontal header for matrix

matrix = []
# row = []
# row.append(0)
# h = tracts.loc[:,'GEOID'].values[:10].tolist() # horizontal header
# row = row + h
# matrix.append(row)

In [37]:
tracts

,GEOID,TRIPS_ORIG,CASES
0,17091011700,27.0,63.068966
1,17091011800,93.0,63.068966
2,17119400951,433.0,46.803279
3,17119400952,4.0,46.803279
4,17135957500,34.0,21.875000
...,...,...,...
3118,17037000100,145.0,45.714286
3119,17037001500,346.0,45.714286
3120,17037000400,13.0,45.714286
3121,17037000300,110.0,45.714286


In [38]:
# populate rest of the rows
risk_flow_matrix = []
num_geoID = tracts.shape[0]
for i in range(num_geoID):
    row = [] # each row in the risk flow matrix
    cases_at_i = tracts.iloc[i, -1]    # case number at i
    origin_at_i = tracts.iloc[i, 0] # residence
    m_work = mRes.get(origin_at_i,{}) # returns the value for the specified key or {} 
    for j in range(num_geoID):       # for each dest at j
        dest_at_j = tracts.iloc[j, 0]    # geoid at dest j
        trips = 0
        if len(m_work) != 0:
            trips = m_work.get(dest_at_j, 0)
        row.append((cases_at_i * trips/total_risk_flow))
    risk_flow_matrix.append(row)

In [39]:
risk_flow_matrix_df = pd.DataFrame(risk_flow_matrix, index = tracts.loc[:,'GEOID'], columns = tracts.loc[:, 'GEOID'])
risk_flow_matrix_df

GEOID,17091011700,17091011800,17119400951,17119400952,17135957500,17119401100,17119401500,17119401722,17189950200,17189950400,...,17037000900,17037001600,17037000500,17037001700,17037001900,17037000100,17037001500,17037000400,17037000300,17037000200
GEOID,,,,,,,,,,,,,,,,,,,,,
17091011700,0.000003,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
17091011800,0.000003,0.000026,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
17119400951,0.000000,0.000000,0.000150,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
17119400952,0.000000,0.000000,0.000022,0.000002,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
17135957500,0.000000,0.000000,0.000000,0.000000,0.000019,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17037000100,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000010,0.000000,0.0,0.0,0.000126,0.000002,0.000000,0.000000,0.000005
17037001500,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000002,0.000002,0.0,0.0,0.000000,0.000036,0.000000,0.000002,0.000000
17037000400,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000005,0.000007,0.0,0.0,0.000000,0.000013,0.000006,0.000007,0.000000


In [40]:
# export the matrix to an excel file
risk_flow_matrix_df.to_csv('risk_flow_matrix.csv')

In [24]:
# sorted(tracts['TRIPS_ORIG'].unique())

[2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 75.0,
 76.0,
 77.0,
 78.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 84.0,
 85.0,
 86.0,
 87.0,
 88.0,
 89.0,
 90.0,
 91.0,
 92.0,
 93.0,
 94.0,
 95.0,
 96.0,
 97.0,
 98.0,
 99.0,
 100.0,
 101.0,
 102.0,
 103.0,
 104.0,
 105.0,
 106.0,
 107.0,
 108.0,
 109.0,
 110.0,
 111.0,
 112.0,
 113.0,
 114.0,
 115.0,
 116.0,
 117.0,
 118.0,
 119.0,
 120.0,
 121.0,
 122.0,
 123.0,
 124.0,
 125.0,
 126.0,
 127.0,
 128.0,
 129.0,
 130.0,
 131.0,
 132.0,
 134.0,
 135.0,
 136.0,
 137.0,
 138.0,
 139.0,
 140.0,
 1